# Lab 1 - FrozenLake MDP: Part 1
# Assignment

- In this assignment you will learn how to tackle problems with limited state spaces.
- In particular we consider the FrozenLake MDP problem.

# Outline

- Part 0 introduces us to [gym](https://gym.openai.com/), an environment that allows us to test our reinforcement learning algorithms in various problems
- In Part 1, you will implement a policy iteration algorithm (HW1)
- In Part 2, you will implement Q-Learning and SARSA (in next homework HW2) 

# Deliverable

Regarding the Lab:

- Make sure your code runs from top to bottom without any errors.
- Your submitted Notebook must contain saved outputs.

In [1]:
!{os.sys.executable} -m pip install numpy
!{os.sys.executable} -m pip install gym

/bin/bash: {os.sys.executable}: command not found
/bin/bash: {os.sys.executable}: command not found


In [2]:
import os
# You will need numpy and gym. You can try running the following lines to install them
# The assignment is tested on Python3.8 so in case you are having installation issues you might 
# want to try installing that version. 


import gym
import numpy as np

# Part 0 - Introduction to Gym
- We look at [FrozenLake-v0 environment](https://gym.openai.com/envs/FrozenLake-v0/) in gym. 
- You don't need to write any code for this part
- you should still understand the code to help you solve Part 1 and Part 2

In [3]:
# Import the environment we will use in this assignment
env=gym.make('FrozenLake-v0') 

# Show the model
print(f"Number of States {env.nS}, Number of Actions {env.nA}")
print(f"Reward range {env.reward_range}")

Number of States 16, Number of Actions 4
Reward range (0, 1)


In [4]:
env.reset() # reset the environment 

0

In [5]:
# visualize the current state
env.render()


SFFF
FHFH
FFFH
HFFG


In [6]:
# run a policy that chooses actions randomly 
env.reset()
n = 25
for i in range(n):
    a = env.action_space.sample() # Sample Random Action
    state, reward, finished, _ = env.step(a)
    if finished: break
        
print(f'Render State after {n} slots')
env.render()
print(f'Reached terminal state? {finished}')

Render State after 25 slots
  (Up)
SFFF
FHFH
FFFH
HFFG
Reached terminal state? True


In [7]:
env.reset() # Let's reset the state again
env.render()


SFFF
FHFH
FFFH
HFFG


# Part 1 - MDP and Planning: Implement Policy Iteration 
- In this part we will focus on methods that assume knowledge of the enivonment dynamics, in partucular you will implement Policy Iteration. 
- The environment model can be obtained through `env.P`

In [8]:
env.P[14]

{0: [(0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False)],
 1: [(0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True)],
 2: [(0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False)],
 3: [(0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False)]}

In [9]:
# No need to change anything here. Try to understand what happens 

# let's look at a random state-action pair and observe its transition characteristics
# you can re-run this cell to get a different state-action pair
random_state  = env.observation_space.sample()
random_action = env.action_space.sample()
# returns a list of tuples (probability,newstate,reward,is_terminal_state)
env.P[random_state][random_action] 

[(0.3333333333333333, 2, 0.0, False),
 (0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [12]:
############################
# YOUR CODE HERE #
# Print all the terminal states in the environment.
# you can use env.P
############################
terminal_states = []
for s in range(env.nS):
    for a in range(env.nA):
        for prob, next_state, reward, isend in env.P[s][a]:
            if isend:
                terminal_states.append(next_state)
print(list(set(terminal_states)))

[5, 7, 11, 12, 15]


In [13]:
# Verify your solution (look at the positions where final states are)
env.render()


SFFF
FHFH
FFFH
HFFG



### Step A: Implement Policy Evaluation


In [14]:
def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
    """Evaluate the value function from a given policy.

    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    policy: np.array[nS]
        The policy to evaluate. Maps states to actions.
    tol: float
        Terminate policy evaluation when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns
    -------
    value_function: np.ndarray[nS]
        The value function of the given policy, where value_function[s] is
        the value of state s
    """
    value_function = np.zeros(nS)          
            
    ############################
    # YOUR IMPLEMENTATION HERE #

    ############################
    def calc_vs(P, policy, value_function, gamma, cur_state):
        result = 0
        total_prob = 0
        action = policy[cur_state]
        for (prob, next_state, reward, isend) in P[cur_state][action]:
            result = result + prob * (reward + gamma * (value_function[next_state]))
            total_prob = total_prob + prob
        assert total_prob == 1
        return result

    converge = False
    while not converge:
        cur_tol = 0
        for s in range(nS):
            previous_value = value_function[s]
            value_function[s] = calc_vs(P, policy, value_function, gamma, s)
            cur_tol = max(cur_tol, abs(value_function[s] - previous_value))
        if cur_tol < tol:
            converge = True

    return value_function

#### Evaluate random policies

In [15]:
# Test your policy_evaluation on 5 randomly generated deterministic policies
# print the value function of the policies

############################
# YOUR CODE HERE #

for i in range(5):
   random_policy = np.random.randint(low=0, high=env.nA, size=env.nS)
   print(f'-------- Policy {i}','-'*30)
   print(policy_evaluation(env.P, env.nS, env.nA, random_policy))

############################

-------- Policy 0 ------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-------- Policy 1 ------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-------- Policy 2 ------------------------------
[0.00883696 0.00425529 0.001908   0.00076179 0.01824949 0.
 0.0005724  0.         0.0344539  0.08058255 0.16411957 0.
 0.         0.23419423 0.54651705 0.        ]
-------- Policy 3 ------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-------- Policy 4 ------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Step B: Implement Policy Improvement

In [16]:
def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
    """
    Given the value function from policy improve the policy.

    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    value_from_policy: np.ndarray
        The value calculated from the policy
    policy: np.array
        The previous policy.

    Returns
    -------
    new_policy: np.ndarray[nS]
        An array of integers. Each integer is the optimal action to take
        in that state according to the environment dynamics and the
        given value function.
    """
    new_policy = np.zeros(nS, dtype='int')
    ############################
    # YOUR IMPLEMENTATION HERE #
    ############################
    def calc_qs(P, value_function, action, gamma, cur_state):
        result = 0
        total_prob = 0
        for (prob, next_state, reward, isend) in P[cur_state][action]:
            result = result + prob * (reward + gamma * (value_function[next_state]))
            total_prob = total_prob + prob
        assert total_prob == 1
        return result

    converge = False
    old_policy = policy.copy()
    while not converge:
        for s in range(nS):
            qs = np.zeros(nA, dtype='float')
            for a in range(nA):
                qs[a] = calc_qs(P, value_from_policy, a, gamma, s)
            new_policy[s] = np.argmax(qs)
        if np.all(old_policy == new_policy):
            converge = True
        else:
            old_policy = new_policy

    return new_policy

In [17]:
# Print the value before and after policy improvements for 5 randomly generated policies

############################
# YOUR CODE HERE #
############################

for i in range(5):
   random_policy = np.random.randint(low=0, high=env.nA, size=env.nS)
   print(f'-------- Policy {i}','-'*30)
   prev_value_function = policy_evaluation(env.P, env.nS, env.nA, random_policy)
   print("Policy : {}".format(random_policy))
   print("Value Function: {}".format(prev_value_function))
   print("Mean : {}".format(prev_value_function.mean()))
   print() # Print value function before improvement

   improved_policy = policy_improvement(env.P, env.nS, env.nA, prev_value_function, random_policy)
   improved_value_function = policy_evaluation(env.P, env.nS, env.nA, improved_policy)
   print(f'-------> Policy {i} IMPROVED','-'*21)
   print("Policy : {}".format(improved_policy))
   print("Value Function: {}".format(improved_value_function))
   print("Mean : {}".format(improved_value_function.mean()))
   print() # Print value function before improvement

-------- Policy 0 ------------------------------
Policy : [0 3 2 3 3 1 2 3 2 0 2 2 0 0 1 0]
Value Function: [0.         0.01449378 0.03497827 0.02487401 0.         0.
 0.05860305 0.         0.         0.         0.16043798 0.
 0.         0.         0.4761904  0.        ]
Mean : 0.04809859299338834

-------> Policy 0 IMPROVED ---------------------
Policy : [1 2 2 3 0 0 0 0 0 1 0 0 0 1 2 0]
Value Function: [0.01203756 0.02316454 0.05516846 0.04008759 0.00606778 0.
 0.09045277 0.         0.00254226 0.14943666 0.24648406 0.
 0.         0.24928898 0.58179579 0.        ]
Mean : 0.09103290282136381

-------- Policy 1 ------------------------------
Policy : [2 1 2 0 2 3 2 3 0 0 0 1 2 0 1 1]
Value Function: [4.24004116e-03 9.75342818e-03 2.89427722e-02 1.22320477e-02
 1.41109920e-03 0.00000000e+00 5.66255857e-02 0.00000000e+00
 5.62416619e-04 1.85751289e-04 1.59904063e-01 0.00000000e+00
 0.00000000e+00 7.27516894e-05 4.76215820e-01 0.00000000e+00]
Mean : 0.04688411105372361

-------> Policy 1 I

### Step C: Implement Policy Iteration

In [18]:
def policy_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
    """ 
    Run policy iteration for dynamics of P.

    You should use your methods: policy_evaluation() and policy_improvement() here

    Parameters: 
    P, nS, nA, gamma: defined at beginning of file
    tolerance:        tolerance parameter used in policy_evaluation()
    
    Returns: 
    value_function: np.ndarray[nS]
    policy:         np.ndarray[nS]
    """

    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)

    ############################
    # YOUR IMPLEMENTATION HERE #
    ############################
    converge = False
    policy = np.random.randint(low=0, high=nA, size=nS)
    prev_policy = policy.copy()
    while not converge:
        value_function = policy_evaluation(P, nS, nA, policy, gamma, tol)
        policy = policy_improvement(P, nS, nA, value_function, policy, gamma)
        if np.all(prev_policy == policy):
            converge = True
        else:
            prev_policy = policy.copy()
    return value_function, policy

In [19]:
def test_policy(policy, n_trials = 100):
    env = gym.make("FrozenLake-v0")
    env.reset()
    success = 0

    for _ in range(n_trials):
        done = False
        state = env.reset()
        while not done:
            action = policy[state]
            state, reward, done, _ = env.step(action)
        success = success + reward

    avg_success_rate = success / n_trials
    return avg_success_rate

In [20]:
random_policy = np.random.randint(low=0, high=env.nA, size=env.nS)
test_policy(random_policy)

0.0

In [21]:
def visualize_policy(policy):
    vpolicy = policy.copy()
    mapper = {0: 'left', 1: 'down', 2: 'right', 3: 'up'}
    vpolicy = [mapper[x] for x in vpolicy.tolist()]
    print(vpolicy[0:4])
    print(vpolicy[4:8])
    print(vpolicy[8:12])
    print(vpolicy[12:16])

#### Call your function for gamma=0.9 and gamma=0.6

In [22]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [23]:
V_pi_s, p_pi_s = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
V_pi_s, p_pi_s

(array([0.06476289, 0.05841569, 0.07252933, 0.05378964, 0.08867795,
        0.        , 0.11137269, 0.        , 0.14325408, 0.24628655,
        0.29886952, 0.        , 0.        , 0.37915415, 0.63865133,
        0.        ]), array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]))

In [24]:
test_policy(p_pi_s)

0.76

In [25]:
visualize_policy(p_pi_s)

['left', 'up', 'left', 'up']
['left', 'left', 'left', 'left']
['up', 'down', 'left', 'left']
['left', 'right', 'down', 'left']


In [26]:
V_pi_f, p_pi_f = policy_iteration(env.P, env.nS, env.nA, gamma=0.6, tol=1e-3)
V_pi_f, p_pi_f

(array([0.00069547, 0.00139264, 0.00562142, 0.00171616, 0.00280064,
        0.        , 0.02169924, 0.        , 0.01214942, 0.04755381,
        0.1032758 , 0.        , 0.        , 0.12348492, 0.44745551,
        0.        ]), array([1, 3, 2, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]))

In [27]:
test_policy(p_pi_f)

0.39

In [28]:
visualize_policy(p_pi_f)

['down', 'up', 'right', 'up']
['left', 'left', 'left', 'left']
['up', 'down', 'left', 'left']
['left', 'right', 'down', 'left']


#### What do you observe in terms of impact of gamma on the actions taken by the policy?

*YOUR ANSWER*

Discount factor / gamma causes the Value Iteration to place more emphasis in the current rewards as opposed to the future rewards. Therefore, this causes the policy to be more aggressive in reaching the goal.